## Query Courses

### To load pickle file

In [1]:
from course import Course
import pickle

# Should take around a minute, pickle file is in Drive linked in README
with open("courses_directory.pkl", "rb") as f: 
    course_directory: dict[str, Course] = pickle.load(f) 

In [2]:
course_directory.__len__()

5970

In [4]:
course_directory['CAS PS 101'].Post

[CAS PS 231,
 CAS NE 234,
 CAS PS 211,
 CAS NE 323,
 CAS PS 222,
 CAS NE 327,
 CAS PS 336,
 CAS PS 241,
 CAS PS 234,
 CAS PS 251,
 CAS PS 261,
 CAS PS 323,
 CAS PS 324,
 CAS PS 325,
 CAS PS 326,
 CAS PS 327,
 CAS PS 354,
 CAS PS 370,
 CAS PS 371,
 CAS PS 560,
 SAR SH 335]

### Find Class

In [61]:
College: str = "CAS" # Three letter acronym like "CAS"
Department: str = "BI" # Two letter acronym like "BI"
Number: int = 203 # Three numbers like "203"

c: Course = course_directory[f"{College} {Department} {Number}"]

### Methods

#### Get Course Name, Code, and Attr

In [62]:
College, Department, Number = c.College, c.Department, c.Number

code: str = c.__repr__() # ex. CAS BI 203 

class_name: str = c.getCourseName() # ex. Cell Biology

#### Get PreReqs and CoReqs and Attr

In [63]:
pre_pres, co_reqs, leads_to = [c.UndergradReq + c.GradReq, \
                                c.UndergradCoReq + c.GradCoReq, c.Post]

# Returns clips with relivant info for course reqs
pre_req_text: dict[str, str] = c.getPreReqsText()

#### Hubs

In [64]:
Hubs: list[str] = c.getHubs()

#### Semesters Offered

In [65]:
semesters: list[str] = c.getSemesters()

#### tmi (to much information)

In [66]:
c.tmi() # returns str

'CAS BI 203 (Cell Biology) = R [CAS BI 108, CAS NE 102, CAS CH 102] C [CAS CH 203] '

#### Webpage and url

In [67]:
from bs4 import BeautifulSoup

website: BeautifulSoup = c.webpage

description: str = c.getDesc() # id='course-content' of course page

url: str = c.url

link: str = c.link # kwarg for c.__init__() for web scraping

valid_link: bool = c.valid_link # False if page not found

## Construct Course

In [ ]:
# Careful with this one, it will overwrite the pickle file and remove the Post list if you provide it the course_directory

name: str = "CAS BI 203"

c: Course = Course(name, link=None, cd={})

## Webscraping

In [ ]:
# Caution does a lot, broken have to declare BUCourseBranch class in same place as course_directory

from main import BUCourseBranch

save_branch = BUCourseBranch()

save_branch.run() # or save_branch.__repr__()

## Other

In [2]:
# To JSON File

from dataclasses import dataclass

@dataclass
class json_comp():

    college: str # ex CAS
    department: str # ex BI
    number: str # ex 203

    name: str # ex Cell Biology

    desc: str # Blurb on webpage

    valid_link: bool # if webpage exists

    uprereqs: list[str] # Undergrad Pre reqs
    gprereqs: list[str] # Grad Pre Reqs

    uprereqstext: str
    gprereqstext: str

    ucoreqs: list[str] # Undergrad Co reqs
    gcoreqs: list[str] # Grad Co Reqs

    ucoreqstext: str
    gcoreqstext: str

    post: list[str] # Classes lead to

    hubs: list[str] # Hubs satisifed

    semesters: list[str] # ['FALL', 'SPNG']
  

to_json: dict[str, json_comp] = {}

for name, _class in zip(course_directory.keys(), course_directory.values()):

    to_json.__setitem__(name, json_comp(
        college=_class.College,
        department=_class.Department,
        number=_class.Number,
        name=_class.getCourseName(),
        desc=_class.getDesc(),
        valid_link=_class.valid_link,
        uprereqs=[c.__repr__() for c in _class.UndergradReq],
        gprereqs=[c.__repr__() for c in _class.GradReq],
        ucoreqs=[c.__repr__() for c in _class.UndergradCoReq],
        gcoreqs=[c.__repr__() for c in _class.GradCoReq],
        uprereqstext=_class.getPreReqsText()['upr'] if 'upr' in _class.getPreReqsText() else "",
        gprereqstext=_class.getPreReqsText()['gpr'] if 'gpr' in _class.getPreReqsText() else "",
        ucoreqstext=_class.getPreReqsText()['ucr'] if 'ucr' in _class.getPreReqsText() else "",
        gcoreqstext=_class.getPreReqsText()['gcr'] if 'gcr' in _class.getPreReqsText() else "",
        post=[c.__repr__() for c in _class.Post],
        hubs=_class.getHubs(),
        semesters=_class.getSemesters()
    ).__dict__)

In [7]:
assert to_json.__len__() == course_directory.__len__()

print(to_json['CAS BI 203'])

{'college': 'CAS', 'department': 'BI', 'number': '203', 'name': 'Cell Biology', 'desc': 'Undergraduate Prerequisites: (CASBI108 OR CASNE102) and CAS CH 102 or equivalent. ; Undergraduate Corequisites: (CASCH203)or equivalent. - Principles of cellular organization and function: biological molecules, flow of genetic information, membranes and subcellular organelles, and cell regulation. Three hours lecture, one hour discussion. Students may receive credit for CAS BI 203 or 213, but not both courses. Effective Fall 2019, this course fulfills a single unit in each of the following BU Hub areas: Scientific Inquiry I, Quantitative Reasoning I, Critical Thinking.', 'valid_link': True, 'uprereqs': ['CAS BI 108', 'CAS NE 102', 'CAS CH 102'], 'gprereqs': [], 'uprereqstext': ' (CASBI108 OR CASNE102) and CAS CH 102 or equivalent. ; ', 'gprereqstext': '', 'ucoreqs': ['CAS CH 203'], 'gcoreqs': [], 'ucoreqstext': ' (CASCH203)or equivalent.', 'gcoreqstext': '', 'post': ['CAS BI 216', 'CAS BI 311', 'CA

In [8]:
import json

with open('course_directory_2.json', 'w') as f:

    f.write(json.dumps(to_json))

In [3]:
class dummy():

    def __init__(self, c):
        self.c = c

    def __repr__(self):
        return self.c
    
    def tmi(self):

        return ""

    def getHubs(self):

        print(self.c)

        return []
    
    def getSemesters(self):

        print(self.c)

        return []

In [4]:
from colors import colors

def load_courses(check_semesters: bool = False) -> list[Course]:
    courses: list[str] = []

    masters = ['MF', 'MS']

    with open(f'subi.txt', 'r') as mc:

        text = mc.read().split('\n')

        

        for semester in text:

            name, info = semester.split(' ? ')

            cs = [course_directory.get(' '.join([i for i in c.split(' ') if i != '']), dummy(' '.join([i for i in c.split(' ') if i != '']))) for c in info.split('|')]

            if check_semesters:

                semester = None
                
                match name[[i for i, a, b in zip(range(name.__len__()), name, name.upper()) if a == b][1]:]:
                    
                    case 'F':

                        semester = 'FALL'

                    case 'S':

                        semester = 'SPRG'

                    case 'Sum':

                        semester =  'SUMMER'

                    case _:

                        raise ValueError(f'semester not found')

                if semester != 'SUMMER':

                    for c in cs:

                        course_sem = c.getSemesters()

                        pre = c.all_reqs

                        if not course_sem:
                            
                            print(f'{colors["yellow"]}{c.__repr__()} unclear {colors["white"]}')

                        elif semester not in course_sem:

                            print(f'{colors["red"]}{c.__repr__()} not in {semester}, {c.getSemesters()}{colors["white"]}')
                        
                        else:

                            print(f'{colors["green"]}{c.__repr__()} in {semester}, {c.getSemesters()}{colors["white"]}')

                        print(f"{set(pre) & set(courses)} {pre}")


            if '*' not in info and name.replace(' ', '') not in masters:

                # f'{Col} {Dep} {Num}' for Col, Dep, Num

                courses.extend(cs)

    return courses

courses = load_courses(check_semesters=True)

CAS LS 111 in FALL, ['SPRG', 'FALL']
set() []
CAS NE 101 in FALL, ['SPRG', 'FALL']
set() []
CAS CH 109 in FALL, ['FALL']
set() []
CAS WR 120 in FALL, ['SPRG', 'FALL']
set() [CAS WR 112]
CAS LS 112 in SPRG, ['SPRG', 'FALL']
{CAS LS 111} [CAS LS 111]
CAS NE 102 in SPRG, ['SPRG']
{CAS WR 120} [CAS WR 120]
CAS CH 110 in SPRG, ['SPRG']
{CAS CH 109} [CAS CH 109]
CAS PS 101 in SPRG, ['SPRG', 'FALL']
set() []
CAS SO 215 in SPRG, ['SPRG', 'FALL']
set() []
CAS LS 211 in FALL, ['SPRG', 'FALL']
{CAS LS 112} [CAS LS 112]
CAS NE 218 in FALL, ['FALL']
{CAS WR 120} [CAS NE 116, CAS CH 116, CAS WR 120]
CAS BI 203 in FALL, ['SPRG', 'FALL']
{CAS NE 102} [CAS BI 108, CAS NE 102, CAS CH 102]
CAS CH 218 in FALL, ['FALL']
set() [CAS CH 116, CAS BI 116, CAS NE 116]
WED HD 400 in FALL, ['FALL']
set() []
CAS LS 212 in SPRG, ['SPRG', 'FALL']
{CAS LS 211} [CAS LS 211]
CAS NE 202 in SPRG, ['SPRG', 'FALL']
{CAS NE 101} [CAS NE 101]
CAS NE 212 in SPRG, ['SPRG', 'FALL']
set() []
CAS CH 204 in SPRG, ['SPRG']
set() [CA

In [5]:
course_directory['WED YJ 360'].Post

[WED YJ 401, WED YJ 403, WED YJ 404]

In [18]:
import numpy as np, re

from collections.abc import Callable

from typing import Tuple, Callable, Union

from colors import colors

search_for = {}

def find_course_by(func: Callable) -> list[Course]:

    return list(filter(func, list(course_directory.values())))

def find_courses_with(hub: str) -> list[Course]:
    return [c for c in course_directory.values() if hub in c.getHubs()]

def course_reports(cs: list[Course]) -> None:
    print(*[f'{c.__repr__()}: {c.getCourseName()} -> {c.getSemesters()} | {c.all_reqs} :: {c.getCredits()} ? {c.getHubs()}' for c in cs], sep='\n')

def hubs_acronym(hubs: Union[list[str], str]) -> list[str]:

    if isinstance(hubs, list):

        return ["".join([w[0] for w in re.split(r'[ /]+', h) if w != '']) for h in hubs]
        
    elif isinstance(hubs, str):

        return "".join([w[0] for w in re.split(r'[ /]+', hubs) if w != ''])

def hubs_report(cs: list[Course]) -> dict[str, int]:

    total = np.unique(sum([c.getHubs() for c in cs], []), return_counts=True)

    print(*[f'{hubs_acronym(name)}:{num}' for name, num in zip(*total, [])], sep='\n')

    hubs = {}

    [hubs.__setitem__(name, num) for name, num in zip(*total)]

    return hubs

def get_match_score(c, hubs) -> int:

    ch = c.getHubs()

    return sum([1 if h in ch else 0 for h in hubs])

def check_hub(hub: Union[list[str], str], num: Union[list[int], int]) -> Callable:

    def check(user_hubs: dict[str, int]) -> bool:

        if isinstance(hub, list):

            return sum([h in user_hubs and int(user_hubs[h]) >= int(num) for h, num in zip(hub, num)]).__bool__()
        
        elif isinstance(hub, str):

            return hub in user_hubs and int(user_hubs[hub]) >= int(num)

        else:

            raise TypeError(f"hub arg should be of type str or list not {type(hub)}")
        
    return check

def load_requirements() -> list[str]:

    with open('hub_requirements.txt', 'r') as hr:

        return hr.read().split('\n')

def check_requirements() -> bool:

    checks = [c for c in [l.split(':') if '||' not in l else np.array([o.split(':') for o in l.split(' || ')]).T.tolist() for l in load_requirements()]]

    print([[hubs_acronym(c[0]), c[1]] for c in checks])

    user_hubs = hubs_report(load_courses())

    for check in checks:

        output: str = f'{":".join([*check])}<={user_hubs.get(check[0], 0)}' if isinstance(check[0], str) else ' || '.join([f'{":".join([*ch])}<={user_hubs.get(ch[0], 0)}' for ch in np.array(check).T.tolist()])

        if check_hub(*check)(user_hubs):

            print(f'{colors["green"]}{output}{colors["white"]}')

        else:

            search_for[check[0]] = int(check[1]) - int(user_hubs.get(check[0], 0))

            print(f'{colors["red"]}{output}{colors["white"]}')


check_requirements()

print(search_for)


[['AE', '1'], ['CI', '2'], ['CT', '2'], ['DME', '1'], ['ER', '1'], ['FWS', '1'], ['GCaIL', '2'], ['HC', '1'], ['OaoSC', '1'], ['PIaLM', '1'], ['QRI', '1'], ['QRI', '1'], ['RaIL', '2'], ['SII', '1'], ['SII', '1'], [['SII', 'SII'], ['1', '1']], ['TC', '2'], ['TIiC', '1'], ['WRaI', '1'], ['WC', '2']]

Aesthetic Exploration:1<=1
Creativity/Innovation:2<=2
Critical Thinking:2<=10
Digital/Multimedia Expression:1<=1
Ethical Reasoning:1<=6
First-Year Writing Seminar:1<=1
Global Citizenship and Intercultural Literacy:2<=2
Historical Consciousness:1<=1
Oral and/or Signed Communication:1<=3
Philosophical Inquiry and Life's Meanings:1<=1
Quantitative Reasoning I:1<=6
Quantitative Reasoning II:1<=2
Research and Information Literacy:2<=6
Scientific Inquiry I:1<=7
Social Inquiry I:1<=7
Scientific Inquiry II:1<=4 || Social Inquiry II:1<=0
Teamwork/Collaboration:2<=5
The Individual in Community:1<=3
Writing, Research, and Inquiry:1<=1
Writing-Intensive Course:2<=3
{}


In [ ]:
course_reports(courses)

In [7]:
choices = [c for c in find_course_by(lambda c: False if c.getCredits() is None else c.getCredits() <= 4 and 2 <= get_match_score(c, list(search_for.keys())))]
choices

[CAS AA 114,
 CAS AA 215,
 CAS AA 237,
 CAS AA 287,
 CAS AA 296,
 CAS AA 297,
 CAS AA 335,
 CAS AA 356,
 CAS AA 371,
 CAS AA 382,
 CAS AA 388,
 CAS AA 514,
 CAS AA 656,
 CAS AH 111,
 CAS AH 112,
 CAS AH 114,
 CAS AH 201,
 CAS AH 210,
 CAS AH 215,
 CAS AH 220,
 CAS AH 240,
 CAS AH 257,
 CAS AH 284,
 CAS AH 325,
 CAS AH 326,
 CAS AH 327,
 CAS AH 333,
 CAS AH 352,
 CAS AH 361,
 CAS AH 365,
 CAS AH 386,
 CAS AH 391,
 CAS AH 392,
 CAS AH 393,
 CAS AH 395,
 CAS AH 398,
 CAS AH 399,
 CAS AH 507,
 CAS AH 727,
 CAS AM 202,
 CAS AM 220,
 CAS AM 505,
 CAS AN 287,
 CAS AN 308,
 CAS AN 319,
 CAS AN 327,
 CAS AN 344,
 CAS AN 375,
 CAS AN 397,
 CAS AN 532,
 CAS AN 565,
 CAS AN 571,
 CAS AN 708,
 CAS AN 719,
 CAS BI 210,
 CAS AN 744,
 CAS AN 775,
 CAS AN 797,
 CAS AR 100,
 CAS AR 331,
 CAS AR 333,
 CAS AR 395,
 CAS AR 565,
 CAS AR 575,
 CAS AR 795,
 CAS BB 592,
 CAS BI 310,
 CAS BI 556,
 CAS BI 586,
 CAS CC 101,
 CAS CC 320,
 CAS CG 357,
 CAS CI 101,
 CAS CI 102,
 CAS CI 200,
 CAS CI 260,
 CAS CI 263,

In [8]:
pairs = []

for i in range(len(choices)):
    for j in range(i+1, len(choices)):

        concat_hubs = (set(choices[i].getHubs()) | set(choices[j].getHubs())) & set(list(search_for.keys()))

        if ((len(concat_hubs) + (1 if 'Creativity/Innovation' in choices[i].getHubs() and 'Creativity/Innovation' in choices[j].getHubs() else 0)) >= 6) and (sum([h not in concat_hubs for h in ['Aesthetic Exploration', 'Digital/Multimedia Expression']]) or (('Creativity/Innovation' in choices[i].getHubs()) ^ ('Creativity/Innovation' in choices[j].getHubs()))):

            print(concat_hubs)

            pair = [choices[i], choices[j]]

            print(pair)

            pairs.append(pair)

{'Historical Consciousness', 'Creativity/Innovation', "Philosophical Inquiry and Life's Meanings", 'Global Citizenship and Intercultural Literacy', 'Aesthetic Exploration'}
[CAS AH 111, CAS LC 261]
{'Historical Consciousness', 'Creativity/Innovation', "Philosophical Inquiry and Life's Meanings", 'Global Citizenship and Intercultural Literacy', 'Aesthetic Exploration'}
[CAS AH 111, CAS LJ 261]
{'Historical Consciousness', 'Creativity/Innovation', "Philosophical Inquiry and Life's Meanings", 'Global Citizenship and Intercultural Literacy', 'Aesthetic Exploration'}
[CAS AH 111, CAS LK 261]
{'Historical Consciousness', 'Creativity/Innovation', "Philosophical Inquiry and Life's Meanings", 'Global Citizenship and Intercultural Literacy', 'Aesthetic Exploration'}
[CAS AH 111, CAS RN 100]
{'Historical Consciousness', 'Creativity/Innovation', "Philosophical Inquiry and Life's Meanings", 'Global Citizenship and Intercultural Literacy', 'Aesthetic Exploration'}
[CAS AH 111, CAS RN 106]
{'Historic

In [9]:
pairs.__len__()

102

In [94]:
course_reports(find_course_by(lambda c: 'Philosophical Inquiry and Life\'s Meanings' in c.getHubs() and c.getCredits() <= 2))

CFA FA 100: Doing, Making & Knowing: The CFA Experience -> ['FALL'] | [] :: 2 ? ["Philosophical Inquiry and Life's Meanings"]
CFA ME 203: Introduction to Music Teaching and Learning -> ['FALL'] | [] :: 2 ? ["Philosophical Inquiry and Life's Meanings"]


In [10]:
course_reports(find_course_by(lambda c: c.getCredits() and c.getCredits() <= 2 and get_match_score(c, search_for)))

CAS BB 498: Honors Research in Biochemistry and Molecular Biology Seminar 2 -> ['SPRG'] | [] :: 1 ? ['Digital/Multimedia Expression']
CAS BI 497: Honors in Biology Seminar -> ['FALL'] | [CAS BI 401, CAS BI 402, CAS BI 453, CAS BB 453] :: 2 ? ['Digital/Multimedia Expression']
CAS BI 498: Honors in Biology Seminar -> ['SPRG'] | [CAS BI 401, CAS BI 402, CAS BI 453, CAS BB 453] :: 2 ? ['Digital/Multimedia Expression']
CAS CC 220: Multimedia Encounters with Core Texts -> [] | [] :: 2 ? ['Digital/Multimedia Expression']
CAS CH 442: Senior Research in Chemistry 2 -> ['SPRG'] | [] :: 2 ? ['Digital/Multimedia Expression']
ENG EK 122: Programming for Engineers -> ['SPRG', 'FALL'] | [] :: 2 ? ['Creativity/Innovation']
CAS WR 202: Children's Literature in the Elementary School -> ['SPRG', 'FALL'] | [] :: 2 ? ['Creativity/Innovation']
CDS DS 280: Spark! UX/UI Design -> [] | [] :: 2 ? ['Digital/Multimedia Expression']
SAR HP 454: Social Determinants of Health - Sex, Sexual Orientation, and Gender ->

In [11]:
ind_choices = set(sum(pairs, []))

In [13]:
course_reports(ind_choices)

CAS LX 660: Historical and Comparative Linguistics -> [] | [] :: 4 ? ['Creativity/Innovation', 'Global Citizenship and Intercultural Literacy', 'Historical Consciousness']
CAS AR 565: Memory in 3-D: Memorials, then and now -> ['FALL'] | [] :: 4 ? ['Creativity/Innovation', 'Digital/Multimedia Expression', 'Historical Consciousness']
CAS HI 343: Taste, Culture, and Power: The Global History of Food -> ['SPRG'] | [] :: 4 ? ['Creativity/Innovation', 'Digital/Multimedia Expression', 'Historical Consciousness']
CAS LX 360: Historical and Comparative Linguistics -> ['SPRG'] | [CAS LX 250] :: 4 ? ['Creativity/Innovation', 'Global Citizenship and Intercultural Literacy', 'Historical Consciousness']
CFA AR 545: Performative Text and Design -> [] | [] :: 4 ? ['Creativity/Innovation', 'Digital/Multimedia Expression', "Philosophical Inquiry and Life's Meanings"]
CAS XL 341: Medieval Travel Writing and the Muslim World -> ['SPRG'] | [] :: 4 ? ['Creativity/Innovation', 'Global Citizenship and Intercu

In [15]:
def getOtherOptions(pairs, choice):

    return set(sum([p for p in pairs if choice in p], [])) - set([choice])

In [16]:
course_reports(getOtherOptions(pairs, course_directory['CFA AR 194']))

CAS LC 261: S24: Chinese Religion -> ['FALL'] | [] :: 4 ? ['Creativity/Innovation', 'Global Citizenship and Intercultural Literacy', "Philosophical Inquiry and Life's Meanings"]
CAS RN 106: Death and Immortality -> ['SPRG', 'FALL'] | [] :: 4 ? ['Creativity/Innovation', 'Global Citizenship and Intercultural Literacy', "Philosophical Inquiry and Life's Meanings"]
CAS RN 100: Introduction to Religion -> ['SPRG', 'FALL'] | [] :: 4 ? ['Creativity/Innovation', 'Global Citizenship and Intercultural Literacy', "Philosophical Inquiry and Life's Meanings"]
CAS LJ 261: Rome and the Chinese World -> [] | [] :: 4 ? ['Creativity/Innovation', 'Global Citizenship and Intercultural Literacy', "Philosophical Inquiry and Life's Meanings"]
CAS LK 261: Rome and the Chinese World -> [] | [] :: 4 ? ['Creativity/Innovation', 'Global Citizenship and Intercultural Literacy', "Philosophical Inquiry and Life's Meanings"]
CAS RN 211: S24: Chinese Religion -> ['FALL'] | [] :: 4 ? ['Creativity/Innovation', 'Global C

In [ ]:
CFA AR 194, CAS RN 106

In [39]:
[f'{c.__repr__()} ({c.getCredits()}): {c.getCourseName()} in {c.getSemesters()} -> {hubs_acronym(c.getHubs())} pre {c.all_reqs}' for c in list(ind_choices)]

["CAS CL 225 (4): Roman Warfare in ['FALL'] -> ['AE', 'CI', 'HC'] pre []",
 "CAS JS 120 (4): The Bible in ['FALL'] -> ['AE', 'CI', 'HC'] pre []",
 "CAS LK 261 (4): Rome and the Chinese World in [] -> ['CI', 'GCaIL', 'PIaLM'] pre []",
 "CAS RN 101 (4): The Bible in ['FALL'] -> ['AE', 'CI', 'HC'] pre []",
 "CAS LF 648 (4): Topics in Text/Image/Spectacle in [] -> ['AE', 'CI', 'HC'] pre [CAS LF 350, CAS LF 351]",
 "CAS LX 660 (4): Historical and Comparative Linguistics in [] -> ['CI', 'GCaIL', 'HC'] pre []",
 "CAS AR 565 (4): Memory in 3-D: Memorials, then and now in ['FALL'] -> ['CI', 'DME', 'HC'] pre []",
 "CAS HI 343 (4): Taste, Culture, and Power: The Global History of Food in ['SPRG'] -> ['CI', 'DME', 'HC'] pre []",
 "CAS LX 360 (4): Historical and Comparative Linguistics in ['SPRG'] -> ['CI', 'GCaIL', 'HC'] pre [CAS LX 250]",
 "CFA AR 545 (4): Performative Text and Design in [] -> ['CI', 'DME', 'PIaLM'] pre []",
 "CAS XL 341 (4): Medieval Travel Writing and the Muslim World in ['SPRG

In [23]:
[f'{c.tmi()} ({c.getCredits()}) {c.getHubs()}' for c in sorted([c for c in course_directory.values() if get_match_score(c, search_for)], key=lambda c: c.getHubs().__len__(), reverse=True)]

["CAS CH 212 (Intensive Organic Chemistry 2) = R [CAS CH 211] C No Co Reqs  (4) ['Creativity/Innovation', 'Oral and/or Signed Communication', 'Research and Information Literacy', 'Scientific Inquiry II']",
 "CAS BI 422 (Biochemistry 2) = R [CAS WR 120, CAS BI 421, CAS CH 421] C No Co Reqs  (4) ['Creativity/Innovation', 'Research and Information Literacy', 'Teamwork/Collaboration', 'Writing-Intensive Course']",
 "CAS BI 622 (Biochemistry 2) = R [CAS WR 120, CAS BI 421, CAS CH 421] C No Co Reqs  (4) ['Creativity/Innovation', 'Research and Information Literacy', 'Teamwork/Collaboration', 'Writing-Intensive Course']",
 "CAS CH 422 (Biochemistry 2) = R [CAS WR 120, CAS BI 421, CAS CH 421] C No Co Reqs  (4) ['Creativity/Innovation', 'Research and Information Literacy', 'Teamwork/Collaboration', 'Writing-Intensive Course']",
 "CAS CH 622 (Biochemistry 2) = R [CAS WR 120, CAS BI 421, CAS CH 421] C No Co Reqs  (4) ['Creativity/Innovation', 'Research and Information Literacy', 'Teamwork/Collabor

In [12]:
course_directory['CDS DS 280'].getDesc()

'User experience design (UX) and user interface engineering (UI) is the design of  user  interfaces and visualization for computer, information, and data products  focusing on  maximizing usability and the user experience. Students complete a  User experience (UX) design encompasses the holistic journey of the end-users  interactions with a company, its services, and its products. UX designers focus  on maximizing usability, accessibility, and the overall user experience. The  course will cover the basic steps of the UX process starting with the discovery  of user insights and leading to a problem definition based around personas,  journey maps, and user stories. Students will then design an application that  responds to this problem by creating low-fidelity wireframes and evolving them  into high fidelity prototypes for user testing. Through this process, students  will complete a series of activities using Figma or similar design tools. While  this course involves developing design a